In [23]:
import pandas as pd

In [24]:
# Got gemeenten from https://organisaties.overheid.nl/Gemeenten
gemeenten_df = pd.read_csv(
    "./Gemeenten.csv",
    sep=";",
    encoding="utf-8",
)

In [25]:
# Extract geolocation
geo_data_col = "Adressen (type, toelichting, straat, huisnummer, toevoeging, postbus, postcode, plaats, regio, provincieAfkorting, land, centroideLatitude, centroideLongitude, centroideRdx, centroideRdy)"

def extract_geolocation(row):
    geo_data = row[geo_data_col].split(";")[0]
    geo_data = geo_data.split(",")

    data = {
        "latitude": None,
        "longitude": None
    }

    for item in geo_data:
        item = item.strip()

        if item.startswith("centroideLatitude"):
            data["latitude"] = float(item.split(":")[1].strip())
        elif item.startswith("centroideLongitude"):
            data["longitude"] = float(item.split(":")[1].strip())

    return pd.Series(data)

# Apply the extraction function to the DataFrame
gemeenten_df[["latitude", "longitude"]] = gemeenten_df.apply(extract_geolocation, axis=1)

In [26]:
# Generate new rows from based on "Bevat plaatsen"
related_towns_col = "Bevat plaatsen"

def extract_related_towns(row):
    related_towns = row[related_towns_col].split(",")
    related_towns = [town.strip() for town in related_towns if town.strip()]

    data = []

    for town in related_towns:
        data.append({
            "Gemeente": row["Afkorting"],
            "Plaats": town,
            "latitude": row["latitude"],
            "longitude": row["longitude"]
        })

    return pd.DataFrame(data)

# Apply the extraction function to create new rows and concatenate all results
related_towns_dfs = gemeenten_df.apply(extract_related_towns, axis=1)
related_towns_df = pd.concat(related_towns_dfs.tolist(), ignore_index=True)

# Add Plaats and Gemeente to gemeenten_df
gemeenten_df["Gemeente"] = gemeenten_df["Afkorting"]
gemeenten_df["Plaats"] = gemeenten_df["Afkorting"]

# Concatenate the original gemeenten DataFrame with the related towns DataFrame
gemeenten_df = pd.concat([gemeenten_df, related_towns_df], ignore_index=True)

In [27]:
# Select relevant columns and rename them
export_cols = [
    "Gemeente",
    "Plaats",
    "latitude",
    "longitude"
]
gemeenten_df = gemeenten_df[export_cols].rename(columns={
    "Gemeente": "gemeente",
    "Plaats": "plaats",
    "latitude": "lat",
    "longitude": "lng"
})

In [28]:
# Save final DataFrame to CSV
gemeenten_df.to_csv(
    "./dutch_places.csv",
    index=False,
    encoding="utf-8-sig"
)